# FlowGPT White Paper
This [project](www.mrdataj.com/chatbot) is an innovative AI-driven Django application that leverages OpenAI's GPT-3.5 model to translate structured text into actionable insights. The application transforms free-text queries into SQL code using a series of chained calls, demonstrating an impressive integration of natural language understanding and code generation. This SQL code is then executed via the Flipside API, a powerful data management platform that runs the queries and returns a resulting table.

The application further capitalizes on the abilities of GPT-3.5 to analyze the results and generate intelligent commentary. These insights are subsequently displayed on the Django frontend, providing users with a clear and comprehensive understanding of the information derived from their original free-text input. This application thus significantly lowers the barrier to data analysis, potentially, in the near future making complex data queries as simple as typing a question in natural language.

## Method
Standard stuff.
- Literature Review
- Examine Existing Frameworks
- Punch Tape
- Incorporate Learning Methods
- Testing and Validation
- Beers

In [22]:
# Requirements.txt
% pip install openai
% pip install flipside


UsageError: Line magic function `%` not found.


In [23]:
# Dependencies
import openai
import os
from flipside import Flipside
from dotenv import load_dotenv
import settings
import json

## OPENAI
My journey began with connecting to OpenAI API and seeing if i can generate a response from a model.

In [24]:
# Load API Key
openai.api_key = os.getenv("OPENAI_API")

In [25]:
# My Hello World OpenAI Function
def ask_openai(message):
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an GPT agent, and I am a human. Ask me anything."},
            {"role": "user", "content": message},
        ]
    )
    
    answer = response.choices[0].message.content.strip()
    return answer

response = ask_openai("The sky green.")
print(response)

I'm sorry, but I believe the sky is actually blue. Can you elaborate on what you meant by "the sky green"?


You win this round AI...
## Which Model?
Many things can be taken into consideration when deciding which model to use from OpenAIs offering. There are various levels of Instruct models than even allow fine tuning. Given there is a large resource of questions with working SQL code answers in the public dashboards on flipside, training these models wouldnt have been a bad experiment if not time consuming one. Chat models on the otherhand promise better conversational replies and more human interactions. 

But ultimately I landed on GPT-3.5-turbo model as its CHEAPER.

### Limitations and Work-arounds
- Dated Knowledge: ChatGPT doesnt know the databases of Flipside and specifically the Flow tables schema. This will have to be taught.
- Token Count: OpenAI API has a limitation to how much data you can give and recieve in one call. 
- Finances: Given i am not funded by CZ (yet at least) i wanted to limit the number of OpenAI API calls i make and the size of these calls.

Bring in model prompts, templates and chaining.


### Model Prompt and Templates
If you've used ChatGPT you'll know the benefit of first giving ChatGPT a role or persona before asking it questions. Same applies here. 

In [26]:
# Test user input message
user_prompt = 'How many transactions where on july 21st 2022?'

# Give GPT the role of a SQL master
system_prompt = 'You are an SQL master. Only you can answer the following questions with strict SQL code, nothing else.'

# Attempt to train it on a sample database schema
db_prompt = open('db_prompt.txt', 'r').read()

# Reiterate snowflake SQL - flipsides db, attach user question to the end
prompt_template = '''
Answer the following questions by writing suggested strict snowSQL code, nothing else. {user_prompt}
'''

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": db_prompt},
        {"role": "assistant", "content": "I now have the db schema, i can write sql code to create a table"},
        {"role": "user", "content": prompt_template.format(user_prompt=user_prompt)},
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7O0zSkfWvXYEaDKqUBuTAs0qvDxcr at 0x2418079f5f0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Assuming that there is a table named `transactions` with a column `transaction_date` that stores the transaction date in the format 'YYYY-MM-DD', the SQL code to count the number of transactions on July 21, 2022 would be:\n\n```\nSELECT COUNT(*) FROM transactions WHERE transaction_date = '2022-07-21';\n```\n\nThis code will return the count of all transactions that occurred on July 21, 2022.",
        "role": "assistant"
      }
    }
  ],
  "created": 1685958034,
  "id": "chatcmpl-7O0zSkfWvXYEaDKqUBuTAs0qvDxcr",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 92,
    "prompt_tokens": 91,
    "total_tokens": 183
  }
}

#### Observations
- Despite asking a couple of times, it never responds in pure SQL code therefore will have to parse the response to pull out the code.
- Inputting just the schema is doesnt know what the information in the tables is so has made some assumptions. Accuracy would improve if i gave it some sample rows, the literature has shown this.

### GPT to SQL
Quick function to parse the output to SQL

In [27]:
# Funtion for converting GPT output to SQL
import re

test_response = {"content": "Assuming that the `BLOCK_TIMESTAMP` column stores the timestamp for each transaction, the following SnowSQL query can be used to count the number of transactions that occurred on July 21st, 2022:\n\n```\nSELECT COUNT(*) \nFROM FACT_TRANSACTIONS\nWHERE CONVERT_TIMEZONE('UTC','America/New_York', BLOCK_TIMESTAMP::TIMESTAMP) BETWEEN '2022-07-21 00:00:00' AND '2022-07-21 23:59:59';\n```\n\nThis query converts the time zone of the `BLOCK_TIMESTAMP` values from UTC to America/New_York to match the date range of interest (July 21st, 2022). It then selects all transactions that occurred during that day and counts them using the `COUNT` function.",}

def gpt2sql(gptstring):
       pattern = r"```(.*)```"
       match = re.findall(pattern=pattern, string=gptstring, flags=re.MULTILINE|re.DOTALL)
       if match:
              sub_string = match[0]
              return sub_string
       
gpt2sql(test_response['content'])

"\nSELECT COUNT(*) \nFROM FACT_TRANSACTIONS\nWHERE CONVERT_TIMEZONE('UTC','America/New_York', BLOCK_TIMESTAMP::TIMESTAMP) BETWEEN '2022-07-21 00:00:00' AND '2022-07-21 23:59:59';\n"

#### Observations
*Chefs kiss*

## Chaining
As mentioned previously, each call has a character limitation. Since we know we get better output by prompting the model with the database schema and sample rows we will hit character limiations pretty quick. There is where chaining API calls comes to play. First i can ask the model which table i should query based on a short list of the tables and a brief description. 

In [29]:
input_prompt = 'How many transactions where on july 21st 2022?'

# settings.tbl_descriptions taken from documentation
# tbl_descriptions = {
#     "flow.core.ez_nft_sales": 'this ez view is the same data table as the prior fact_nft_sales view. nft market sales on the flow blockchain. currently supported in this ez view are transactions that interact with the primary nft marketplace contract (), the fabricant nft marketplace contract, and nba topshot primary market sales. more markets can and will be added.'
#     ,"flow.core.ez_bridge_transactions" : 'this ez_ view is the same data table as the prior fact_bridge_transactions view. this table parses transactions where tokens are bridged to or from the flow network using blocto teleport or the celer bridge.'
#     ,"flow.core.ez_staking_actions" : 'this table provides transaction-level info on flow staking activities.'
#     ,"flow.core.ez_swaps" : 'this table records asset swaps on the flow blockchain.'
#     ,"flow.core.ez_token_transfers" : 'this table records all token transfers on the flow blockchain.'
#     ,"flow.core.fact_transactions" : 'this table records all the transactions of the flow blockchain.'
# }
def ask_which_table(message):
    system_prompt = "You are a blockchain SQL master, you know everything about flow blockchain and can answer any enquiry."
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(settings.tbl_descriptions, indent = 4)  + "\n\n" + message},
            {"role": "assistant", "content": "I now have the table descriptions, i can advise which tables will be best to query"},
            {"role": "user", "content": "Out of the tables mentioned, which table should i query? You answer should ONLY include the table name in the format 'core.flow.?' where the ? is the table name\n\nTable Name:"},
        ]
    )
    string = response.choices[0].message.content
    table = re.findall(pattern=r"flow\.core\.([\w]+)", string=string, flags=re.MULTILINE|re.DOTALL)
    return "flow.core." + table[0]

table_name = ask_which_table(input_prompt)
print(table_name)


flow.core.fact_transactions


Boom. Lets keep the chain, chaining on. From the selected table throw her back in with the correct schema and sample data, and lets see what SQL we get.

In [30]:
def ask_for_sql(message, table_name, check_sql=None):
    system_content=f"Given the SQL table:\n\n{settings.tbl_schema[table_name]}\n\nand sample data\n\n{settings.tbl_data[table_name]}"
    user_content = 'You are an SQL master. Only you can answer the following questions with strict snowflake sql code, nothing else. dont forget to name columns.'
    assistant_content = "I now have the db schema, i can write sql code to create a table"
    if check_sql is None:
        question_content = f"Write strict snowflake sql code to answer the following: {message}"
    else:
        question_content = f"Rewrite strict snowflake sql code to answer the following, question {message} \n\n {check_sql}"

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": question_content},
        ]
    )
    string = response.choices[0].message.content
    if gpt2sql(string) is not None:
        return gpt2sql(string)
    else:
        return string
    
sql_draft=ask_for_sql(input_prompt, table_name)
sql=ask_for_sql(input_prompt, table_name, sql_draft)

### Observations
Added a little check_sql prompt so i could feed the sql back in and see if it comes with the same response. Literature told me this is a good idea and drastically reduces GPT making up table names, columns etc.

Otherwise appears to be working well!

## Flipside
Now we have working SQL code, we need to feed it back to flipside to query the actual data.


In [31]:
from flipside import Flipside 

def ask_flipside(sql):
    # My first query
    load_dotenv()
    FLIPSIDE_API_KEY = os.getenv('FLIPSIDE_API_KEY')

    # Initialize `Flipside` with your API Key and API Url
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    # print('flipside query: ' + query_result_set.status)
    return query_result_set

result = ask_flipside(sql)
table = {'columns': result.columns, 'rows': result.rows}
print(table)

{'columns': ['num_transactions', '__row_index'], 'rows': [[671426, 0]]}


## Putting it all together!

In [32]:
# Simple function asking the model to comment on our output with a given question
def ask_comments(message, table_result):
    system_content=f"Given the question:\n\n{message}\n\nand answer\n\n{table_result}"
    user_content = 'You blockchain business analyts.'
    assistant_content = "I now have the information i need to write comments"
    question_content = f"Rewrite the answer table in words to explain the answer to the question"

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": question_content},
        ]
    )
    string = response.choices[0].message.content
    return string

In [33]:
# The master chain
def llm_chain(user_prompt):
    logs = []
    table =  {
        "columns":'',
        'rows':''
        }
    logs.append('Asking GPT3 to decide which table...')
    try:
        table_name = ask_which_table(user_prompt)
        logs.append('GPT3 decided on table: ' + table_name )
    except:
        logs.append('GPT3 could not decide on table')
        response = 'error'
        return response, logs, table, 
    
    logs.append('Asking GPT3 to write SQL code...')
    try:
        sql = ask_for_sql(user_prompt, table_name)
        logs.append('GPT3 wrote SQL code: ' + sql )
    except:
        logs.append('GPT3 could not write SQL code')
        response = 'error'
        return response, logs, table
    
    logs.append('Asking GTP3 to check the SQL code...')
    try:
        sql = ask_for_sql(user_prompt, table_name, sql)
        logs.append('GPT3 checked SQL code: ' + sql)
    except:
        logs.append('GPT3 could not check SQL code')
        response = 'error'
        return response, logs, table
    
    logs.append('Asking Flipside to run the SQL code...')
    try:
        result = ask_flipside(sql)
        logs.append('Flipside ran the SQL code and got this result: ' + result.status )
    except:
        logs.append('Flipside could not run the SQL code')
        response = 'error'
        return response, logs, table
    try:
        response = ask_comments(user_prompt, result)
        logs.append('GPT3 wrote comments: ')
    except:
        logs.append('GPT3 could not write comments')
        response = 'error'
        return response, logs, table
    table =  {
        "columns":result.columns,
        'rows':result.rows
        }
    return response, logs, table       

input_prompt = "How many transactions where on july 21st 2022?"
response, logs, table = llm_chain(input_prompt)
print(logs, '/n', response, '/n', table)

['Asking GPT3 to decide which table...', 'GPT3 decided on table: flow.core.fact_transactions', 'Asking GPT3 to write SQL code...', "GPT3 wrote SQL code: SELECT COUNT(*) as num_transactions\nFROM flow.core.FACT_TRANSACTIONS\nWHERE DATE_TRUNC('day', BLOCK_TIMESTAMP) = '2022-07-21';", 'Asking GTP3 to check the SQL code...', "GPT3 checked SQL code: SELECT COUNT(*) AS num_transactions\nFROM flow.core.FACT_TRANSACTIONS\nWHERE BLOCK_TIMESTAMP BETWEEN '2022-07-21 00:00:00' AND '2022-07-21 23:59:59';", 'Asking Flipside to run the SQL code...', 'Flipside ran the SQL code and got this result: QUERY_STATE_SUCCESS', 'GPT3 wrote comments: '] /n The query result shows that there were 671,426 transactions that occurred on July 21st, 2022. The result was obtained within 7 seconds of executing the query. There was only one row of data returned by the query. No errors were encountered during the execution of the query. /n {'columns': ['num_transactions', '__row_index'], 'rows': [[671426, 0]]}


## Conclusion
Indeed, the FlowGPT AI chatbot shows promising potential. It has demonstrated its capability to handle complex queries and deliver precise responses based on specific data tables. However, to enhance its performance and extend its applicability, it does require additional testing and an expansion of training data.

Incorporating a list of one-shot prompts can expedite responses for frequently asked questions or common queries. This could improve user experience and make the chatbot more efficient.

Furthermore, while the current model doesn't explicitly handle join operations, its underlying architecture and training show that it's capable of formulating such operations. During the initial testing, the model was able to generate queries involving joins, showing its potential to handle more complex data requests.

In conclusion, while the FlowGPT AI chatbot is already a powerful tool for querying Flow data, there is scope for further improvement and expansion. With continued development and refinement, it could become an even more valuable resource for data analytics.

## References
[Paper:Evaluating the Text-to-SQL Capabilities of Large Language Models](https://arxiv.org/abs/2204.00498)

[Blog: Fine-Tuning GPT-3 for Natural Language to SQL Conversion: An In-Depth Guide](https://blog.futuresmart.ai/fine-tuning-gpt-3-for-natural-language-to-sql-conversion-an-in-depth-guide#heading-gpt-3-parameters)

[Documentation: LangchainSQL](https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html)